## Importing necessary lirabries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

## Taking user input

In [2]:
p1 = float(input("Enter body temperature: "))
p2 = str(input("Enter breed type: "))
p3 = float(input("Enter milk production: "))
p4 = int(input("Enter respiratory rate: "))
p5 = int(input("Enter walking capacity: "))
p6 = float(input("Enter sleeping duration: "))
p7 = int(input("Enter body condition score: "))
p8 = int(input("Enter heart rate: "))
p9 = float(input("Enter eating duration: "))
p10 = float(input("Enter lying down duration: "))
p11 = float(input("Enter ruminating: "))
p12 = int(input("Enter rumen fill: "))
p13 = str(input("Enter faecal consistency: "))

Enter body temperature:  39
Enter breed type:  Normal Breed
Enter milk production:  20.2
Enter respiratory rate:  37
Enter walking capacity:  11989
Enter sleeping duration:  3.9
Enter body condition score:  3
Enter heart rate:  63
Enter eating duration:  3.4
Enter lying down duration:  13.7
Enter ruminating:  6
Enter rumen fill:  3
Enter faecal consistency:  ideal


## Making dictionary of the user input

In [3]:
dict = {'body_temperature': [p1], 'breed_type': [p2], 'milk_production': [p3], 'respiratory_rate': [p4], 'walking_capacity': [p5], 
        'sleeping_duration': [p6], 'body_condition_score': [p7], 'heart_rate': [p8], 'eating_duration': [p9], 'lying_down_duration': [p10], 
        'ruminating': [p11], 'rumen_fill': [p12], 'faecal_consistency': [p13]}

## Making a dataframe of dictionary

In [4]:
df = pd.DataFrame(dict)
df

,body_temperature,breed_type,milk_production,respiratory_rate,walking_capacity,sleeping_duration,body_condition_score,heart_rate,eating_duration,lying_down_duration,ruminating,rumen_fill,faecal_consistency
0,39.0,Normal Breed,20.2,37,11989,3.9,3,63,3.4,13.7,6.0,3,ideal


## Data Preprocessing

In [5]:
df['faecal_consistency']

0    ideal
Name: faecal_consistency, dtype: object

In [6]:
def change(x):
    if x == 'Black faece':
        x = 'Black faeces'
    elif x == 'Fresh blood in faece':
        x = 'Fresh blood in faeces'
    elif x == 'very liquid faece':
        x = 'very liquid faeces'
    return x

faecal_consistency_update = df['faecal_consistency']
t = faecal_consistency_update.map(change)
df['faecal_consistency'] = t

In [7]:
df['body_condition_score'] = df['body_condition_score'].astype(str)

Applying One Hot Encoding on categorical columns

In [8]:
dummies = pd.get_dummies(df[['breed_type', 'body_condition_score', 'faecal_consistency']] , dtype='int')
df = pd.concat([df , dummies] ,axis=1)
df.drop(['breed_type', 'body_condition_score', 'faecal_consistency'] , axis=1 , inplace=True)

## Making an empty dataframe with all possible value of categorical field after one hot encoding

This step is required because we have to match the dimensionality(number of columns) of dataset which is used in Nueral Network model.

In [9]:
columns = ['body_temperature', 'milk_production', 'respiratory_rate',
       'walking_capacity', 'sleeping_duration', 'heart_rate',
       'eating_duration', 'lying_down_duration', 'ruminating', 'rumen_fill',
       'breed_type_Cross Breed', 'breed_type_Normal Breed',
       'body_condition_score_1', 'body_condition_score_2',
       'body_condition_score_3', 'body_condition_score_4',
       'body_condition_score_5',
       'faecal_consistency_Black faeces',
       'faecal_consistency_Fresh blood in faeces',
       'faecal_consistency_extremely firm', 'faecal_consistency_ideal',
       'faecal_consistency_very liquid faeces']

An empty dataframe.

In [10]:
model_df = pd.DataFrame(columns = columns)

Fill the values in those columns which exist in dataframe 'df'.

In [11]:
model_df = model_df.reindex(df.index)
model_df[df.columns] = df

In [12]:
model_df

,body_temperature,milk_production,respiratory_rate,walking_capacity,sleeping_duration,heart_rate,eating_duration,lying_down_duration,ruminating,rumen_fill,...,body_condition_score_1,body_condition_score_2,body_condition_score_3,body_condition_score_4,body_condition_score_5,faecal_consistency_Black faeces,faecal_consistency_Fresh blood in faeces,faecal_consistency_extremely firm,faecal_consistency_ideal,faecal_consistency_very liquid faeces
0,39.0,20.2,37,11989,3.9,63,3.4,13.7,6.0,3,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN


Filling Nan values with 0.

In [13]:
model_df = model_df.fillna(0)

C:\Users\dell\AppData\Local\Temp\ipykernel_31440\213220335.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  model_df = model_df.fillna(0)


In [14]:
scaler = StandardScaler()
scaled = scaler.fit_transform(model_df)
scaled = scaler.transform(model_df)

## Loading the model

In [15]:
model = joblib.load('Cow Health Predictor Final.pkl')

In [16]:
predicted_health_status = model.predict(scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [17]:
predicted_health_status

array([[0.53181064, 0.4681893 ]], dtype=float32)

In [18]:
result = ""
if(np.argmax(predicted_health_status) == 1):
    result = "healthy" 
    print(result)
else:
    result = "unhealthy"
    print(result)

unhealthy
